####Libraries
***

In [1]:
import sklearn
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
# pip install TA-lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz

!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=$HOME
!make
!make install

!TA_LIBRARY_PATH=~/lib TA_INCLUDE_PATH=~/include pip install ta-lib
import talib as ta
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 5.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
--2022-09-16 13:05:59--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-09-16 13:05:59--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownlo

####Load Dataset
***

In [2]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DRE","DXC","META","FTV",   
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]
  #άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-03-01','2022-07-01')['Adj Close'].index)
#παίρνω τιμές για ένα χρόνο και 3 μήνες
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-03-01','2022-07-01')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [3]:
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,101.758362,72.935371,123.264778,109.157562,120.793564,223.396744,157.307007,169.428787,174.066833,136.349152,...,82.900162,273.630005,232.818268,33.455544,118.117966,326.841461,302.489990,214.501114,165.157501,52.391731
2021-03-02,101.232719,72.688164,123.596603,110.222755,116.100159,220.188354,154.726501,170.577332,173.244690,136.879944,...,83.274338,269.190002,232.127426,32.895821,115.247551,326.831726,293.420013,213.649551,164.504959,52.085182
2021-03-03,100.162674,72.564560,124.962944,109.597328,114.064697,218.140457,150.250000,173.620956,174.624710,140.163101,...,83.601753,255.059998,232.424896,31.874580,108.791573,325.103027,294.929993,212.738586,162.952637,52.503201
2021-03-04,99.167709,73.448807,125.001991,107.887161,106.950470,211.850494,148.878494,170.414612,168.527237,140.251556,...,82.853386,239.070007,229.978073,31.324682,109.741829,326.021088,288.600006,209.421509,159.828247,54.537563
2021-03-05,100.153275,75.017616,128.749695,110.730927,112.098396,216.784943,150.022995,170.031769,173.499176,144.822464,...,84.808502,239.050003,236.905823,32.345924,110.829269,339.000977,300.959991,213.293106,166.225342,56.599789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,150.928238,92.870499,172.150925,99.170822,97.111656,248.137924,116.459999,193.575409,164.681396,145.605087,...,96.279518,77.680000,281.488770,35.017067,77.258453,494.089691,248.289993,205.146637,172.702087,86.063782
2022-06-27,151.720825,94.239746,172.300293,98.812370,97.361015,246.087448,113.220001,197.166199,166.274918,143.752029,...,95.824852,75.940002,281.329712,35.186615,76.072624,504.078339,251.990005,203.200089,173.810226,88.173286
2022-06-28,151.076859,94.338966,172.290344,97.975998,94.777618,242.633484,107.400002,195.470001,163.456360,141.659866,...,96.546379,71.820000,278.745331,34.418652,72.525093,506.849640,247.990005,197.769714,168.658813,90.619514


In [4]:
rd1= rd.pct_change()
rd1

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-02,-0.005166,-0.003389,0.002692,0.009758,-0.038855,-0.014362,-0.016404,0.006779,-0.004723,0.003893,...,0.004514,-0.016226,-0.002967,-0.016730,-0.024301,-0.000030,-0.029984,-0.003970,-0.003951,-0.005851
2021-03-03,-0.010570,-0.001700,0.011055,-0.005674,-0.017532,-0.009301,-0.028932,0.017843,0.007966,0.023986,...,0.003932,-0.052491,0.001281,-0.031045,-0.056018,-0.005289,0.005146,-0.004264,-0.009436,0.008026
2021-03-04,-0.009933,0.012186,0.000312,-0.015604,-0.062370,-0.028834,-0.009128,-0.018467,-0.034918,0.000631,...,-0.008952,-0.062691,-0.010527,-0.017252,0.008735,0.002824,-0.021463,-0.015592,-0.019174,0.038747
2021-03-05,0.009938,0.021359,0.029981,0.026359,0.048134,0.023292,0.007687,-0.002247,0.029502,0.032591,...,0.023597,-0.000084,0.030124,0.032602,0.009909,0.039813,0.042827,0.018487,0.040025,0.037813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,0.019338,0.012877,0.020968,0.028819,0.043739,0.040616,0.035752,-0.006820,0.063412,0.031768,...,0.053195,0.052432,0.040260,0.028111,0.047986,-0.008343,0.052790,0.045108,0.011815,0.019833
2022-06-27,0.005251,0.014744,0.000868,-0.003614,0.002568,-0.008263,-0.027821,0.018550,0.009676,-0.012727,...,-0.004722,-0.022400,-0.000565,0.004842,-0.015349,0.020216,0.014902,-0.009489,0.006416,0.024511
2022-06-28,-0.004244,0.001053,-0.000058,-0.008464,-0.026534,-0.014036,-0.051404,-0.008603,-0.016951,-0.014554,...,0.007530,-0.054253,-0.009186,-0.021825,-0.046633,0.005498,-0.015874,-0.026724,-0.029638,0.027743


In [5]:
#prepare the ranks for next month (22 working days later)
rd1_next_month = rd1.shift(-22)
rd1_next_month

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,0.013391,-0.000354,-0.020317,0.002634,0.053960,-0.001504,0.012696,-0.015264,-0.008262,-0.011324,...,-0.010643,0.026634,-0.025483,0.014741,-0.003904,-0.003989,-0.011704,-0.000802,0.004206,-0.015170
2021-03-02,0.002772,0.009917,0.003738,0.016956,0.059281,0.012734,0.021628,0.012953,0.024068,0.022129,...,0.017673,0.019354,0.013317,0.011912,0.005004,-0.013438,0.007288,0.024229,0.017658,0.027942
2021-03-03,-0.021931,0.008768,0.003444,0.017847,0.010811,0.012191,0.020794,-0.004869,0.015313,0.010306,...,0.000431,0.014058,-0.000398,0.016078,0.020496,0.008472,-0.000543,0.010053,0.009677,-0.015508
2021-03-04,-0.007161,0.005562,0.004272,0.011305,-0.024537,-0.007764,-0.000902,0.002742,0.007122,0.000616,...,0.003127,0.008645,0.010239,-0.002826,0.004228,-0.015695,-0.018400,-0.001781,-0.003635,-0.002832
2021-03-05,-0.001613,-0.000922,0.015202,-0.001065,-0.002866,0.007021,0.017237,0.006863,-0.008892,0.006705,...,0.009781,0.009519,0.008597,-0.022953,-0.000729,0.003485,-0.010540,0.002836,-0.004975,0.004437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#first: ranks assigned in order they appear in the array
rd2=rd1_next_month.rank(axis=1,method='first')
rd2=rd2.dropna()
rd2.astype(int)
jk=rd2.copy()
jk.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,41,28,5,31,45,25,40,7,19,14,...,16,44,2,42,23,22,13,27,34,8
2021-03-02,13,21,15,29,45,24,36,25,41,37,...,32,33,26,23,16,3,17,42,31,43
2021-03-03,2,18,13,37,22,24,39,5,33,21,...,11,30,9,34,38,17,7,20,19,3
2021-03-04,14,36,31,41,1,12,22,28,38,26,...,29,39,40,19,30,6,5,21,16,18
2021-03-05,14,18,42,17,11,35,44,34,4,33,...,39,38,37,1,19,27,2,26,8,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-23,13,6,15,20,33,31,28,3,43,23,...,41,39,30,19,36,1,40,34,5,14
2022-05-24,30,39,20,17,23,10,1,41,36,6,...,14,2,19,27,5,43,40,8,32,45
2022-05-25,36,40,39,31,11,23,3,30,17,22,...,43,1,29,15,4,42,20,10,9,44


In [7]:
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
r5=rd2.copy()
rd3=rank_data(r5)
rd3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,5,4,1,4,5,3,5,1,3,2,...,2,5,1,5,3,3,2,3,4,1
2021-03-02,2,3,2,4,5,3,4,3,5,5,...,4,4,3,3,2,1,2,5,4,5
2021-03-03,1,2,2,5,3,3,5,1,4,3,...,2,4,1,4,5,2,1,3,3,1
2021-03-04,2,4,4,5,1,2,3,4,5,3,...,4,5,5,3,4,1,1,3,2,2
2021-03-05,2,2,5,2,2,4,5,4,1,4,...,5,5,5,1,3,3,1,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-23,2,1,2,3,4,4,4,1,5,3,...,5,5,4,3,4,1,5,4,1,2
2022-05-24,4,5,3,2,3,2,1,5,4,1,...,2,1,3,3,1,5,5,1,4,5
2022-05-25,4,5,5,4,2,3,1,4,2,3,...,5,1,4,2,1,5,3,2,1,5


In [8]:
#for absolute rank feature to use 
rd2_feature=rd1.rank(axis=1,method='first')
rd2_feature=rd2_feature.dropna()
rd2_feature.astype(int)
jk_feature=rd2_feature.copy()
jk_feature.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-02,25,30,39,45,1,12,10,42,26,40,...,41,11,33,9,4,37,3,28,29,21
2021-03-03,16,28,36,24,11,18,6,39,33,43,...,31,2,30,4,1,25,32,27,17,34
2021-03-04,24,40,32,19,2,6,25,16,5,33,...,26,1,23,17,37,34,11,20,12,45
2021-03-05,8,16,28,23,43,19,6,1,27,32,...,20,3,29,33,7,37,41,13,39,34
2021-03-08,9,34,28,41,1,22,7,43,42,33,...,38,2,29,8,45,24,19,37,35,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,13,6,15,20,33,31,28,3,43,23,...,41,39,30,19,36,1,40,34,5,14
2022-06-27,30,39,20,17,23,10,1,41,36,6,...,14,2,19,27,5,43,40,8,32,45
2022-06-28,36,40,39,31,11,23,3,30,17,22,...,43,1,29,15,4,42,20,10,9,44


In [9]:
r5_feature=rd2_feature.copy()
rd3_feature=rank_data(r5_feature)
rd3_feature

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-02,3,4,5,5,1,2,2,5,3,5,...,5,2,4,1,1,5,1,4,4,3
2021-03-03,2,4,4,3,2,2,1,5,4,5,...,4,1,4,1,1,3,4,3,2,4
2021-03-04,3,5,4,3,1,1,3,2,1,4,...,3,1,3,2,5,4,2,3,2,5
2021-03-05,1,2,4,3,5,3,1,1,3,4,...,3,1,4,4,1,5,5,2,5,4
2021-03-08,1,4,4,5,1,3,1,5,5,4,...,5,1,4,1,5,3,3,5,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,2,1,2,3,4,4,4,1,5,3,...,5,5,4,3,4,1,5,4,1,2
2022-06-27,4,5,3,2,3,2,1,5,4,1,...,2,1,3,3,1,5,5,1,4,5
2022-06-28,4,5,5,4,2,3,1,4,2,3,...,5,1,4,2,1,5,3,2,1,5


####Functions
***

In [10]:
def RPS_calculation(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    # max_date = np.max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    # print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

In [11]:
def create_historical(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

In [45]:
def predictions(model,assets,stock,k):
  prob=dict()

  for j in assets:
      tmp=[]
      df=stock[j]
      df1=df.dropna()
      X= df1.iloc[:,5:k]
      # print(X) #it has all the days
      y= df1['Rank']
      X_train= X.iloc[:-82] #-43 for every month 82
      
      Y_train= y.iloc[:-82] #-43 for every month ----> change back if you want to run again

      print(Y_train.tail(1))
      model.fit(X_train,Y_train)
      #now each day we will use the models to predict 
      flag=True
      for i in range(61,0,-1): #from 4/01 and later -->predictions 22 telos vazw 61
        if(flag):
          first=X.iloc[-(i+1):-i].index
          print('this is the first day of predictions', first)
          flag=False
        X_validation=X.iloc[-(i+1):-i] 
        # print(X_validation)
        Y_validation= y.iloc[-(i+1):-i]
        # print(Y_validation.index) 
        probability= model.predict_proba(X_validation)
        # print(probability[0])
        tmp.append(probability[0])
      X_validation=X.tail(1) 
      last=X_validation.index
      print('this is last for prediction',last)
      # print(X_validation)
      Y_validation= y.tail(1)
      print(Y_validation.index) 
      probability= model.predict_proba(X_validation)
      print(probability[0])
      tmp.append(probability[0])
      prob[j]=tmp
  return prob , first , last 

In [13]:
def RPS_data(prob_final,rd):
  RPS=pd.DataFrame()

  # RPS=RPS.drop('date',axis=1)
  # print(RPS.index[2])
  for k in prob_final.index: #for everyday of a stock->index
    prob=pd.DataFrame()
    prob1=pd.DataFrame()
    predic=pd.DataFrame()
    if(prob_final['date'][k]!=prob_final['date'].tail(1).values): #if it's not for the last day 
      #for historical data of two days 
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==prob_final['date'][k+1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob : ',prob)
      mask1 = (rd.loc[(rd.index.date>=prob_final['date'][k+1])].index)
      prob1_table = rd.loc[mask1]
      prob1 = prob1_table.shift(-21).head(1)
      print('prob1 : ',prob1)

      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)
      # print(RPS[RPS.index==k])

      
    else:
      #for the last day
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob teleutaio :', prob)
      prob1 = pd.DataFrame(prob_table.iloc[1]).T
      print('prob1 : ',prob1)
      
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      # print(hist_data)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)              
  return RPS

In [14]:
def create_RPS(prob_final,RPS):
  d=prob_final['date']
  name=prob_final.columns[:-1]
  # print(np.split(name.values,len(name)))
  # print(d.values)
  RPS['date']=d.values
  RPS=RPS.set_index('date')
  RPS.columns=assets
  return RPS

####**For 6 Mar.-1 Apr.** 

#####Features
***

In [16]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-03-01','2022-04-02') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [17]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#####Gradient Boosting (baseline) 
***

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
baseline= GradientBoostingClassifier()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-01-31    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.13364068 0.03235837 0.14987794 0.11121756 0.57290545]
Date
2022-01-31    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.11385806 0.21868364 0.30599605 0.19987149 0.16159075]
Date
2022-01-31    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-03  159.814133  99.930481  183.961975  110.589905  126.775017   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-03  294.668579  163.559998  252.387024  173.579086  185.944336  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-03  115.603111  116.669998  303.419739  34.615841  89.954269   

                   UNH    URI           V        VRSK        XOM  
Date                                                              
2022-03-03  509.170868  354.0  225.529037  213.353729  81.485481  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-03-04  159.038071  99.08387  180.370453  110.371742  128.794907   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-04  294.787476  168.346497  248.984695  173.113022  186.431122  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-04  113.630821  121.860001  293.727295  34.894279  92.495735   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-04  506.618011  349.899994  226.903992  216.233383  81.524696  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-31  146.499664  96.603073  179.417999  113.822723  112.406982   

                   AMP   AMZN         AVB         AVY         AXP  ...  \
Date                                                               ...   
2022-03-31  266.332184  124.5  219.365616  177.753952  170.980133  ...   

                   PRU       PYPL          RE       ROL        ROST  \
Date                                                                  
2022-03-31  105.849098  91.529999  274.105255  33.01482  100.586761   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-31  497.807159  307.029999  210.753479  201.466431  84.710785  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-01  148.213638  97.046082  181.640396  114.437546  113.232849   

                  AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                   ...   
2022-04-01  269.58194  124.253502  220.645218  179.657928  171.468307  ...   

                   PRU       PYPL          RE       ROL       ROST  \
Date                                                                 
2022-04-01  107.137924  90.709999  276.180115  32.52755  99.723045   

                   UNH         URI           V        VRSK       XOM  
Date                                                                  
2022-04-01  496.287384  312.450012  207.794388  199.403824  86.45578  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \


,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-02,0.136676,0.062910,0.209967,0.097291,0.361741,0.526390,0.272005,0.023791,0.356796,0.188893,...,0.575020,0.217936,0.202048,0.608200,0.009408,0.051363,0.109825,0.426887,0.092992,0.121743
2022-03-03,0.107302,0.303384,0.287421,0.235072,0.100432,0.004606,0.362518,0.044921,0.073468,0.207482,...,0.571721,0.210497,0.334171,0.171946,0.296721,0.110602,0.266479,0.181466,0.324597,0.019816
2022-03-04,0.148355,0.016207,0.130237,0.081012,0.360211,0.103195,0.157493,0.142611,0.259501,0.178029,...,0.124604,0.042298,0.097799,0.032512,0.093846,0.416537,0.786457,0.198960,0.215115,0.027401
2022-03-07,0.151951,0.199572,0.221118,0.398258,0.177496,0.357028,0.307127,0.259661,0.093194,0.260564,...,0.122490,0.002313,0.185781,0.093089,0.190427,0.603330,0.178123,0.222815,0.152685,0.132665
2022-03-08,0.015286,0.519693,0.021638,0.102358,0.459585,0.204511,0.138045,0.322791,0.163707,0.246146,...,0.704310,0.172844,0.193307,0.233194,0.109841,0.231578,0.364055,0.201920,0.091570,0.081108
2022-03-09,0.127098,0.572876,0.059777,0.161812,0.550702,0.312114,0.032181,0.167454,0.139553,0.152790,...,0.101732,0.156955,0.012917,0.157891,0.208574,0.387751,0.076807,0.114277,0.348552,0.180402
2022-03-10,0.395290,0.102083,0.231601,0.154113,0.215069,0.223932,0.086650,0.071377,0.380790,0.589464,...,0.117858,0.132161,0.104152,0.470169,0.163930,0.000914,0.327139,0.064199,0.164423,0.234420
2022-03-11,0.329373,0.391407,0.017982,0.113859,0.004824,0.000517,0.095681,0.137015,0.195194,0.156346,...,0.318821,0.127306,0.392087,0.153576,0.592832,0.015813,0.377148,0.224481,0.177910,0.187251
2022-03-14,0.210873,0.512020,0.144572,0.177209,0.269388,0.577324,0.109042,0.422282,0.143069,0.188273,...,0.030919,0.275674,0.001610,0.166646,0.151938,0.020719,0.041120,0.252336,0.388798,0.080410


In [19]:
print(RPS1.mean(axis=1).mean())

0.21122327583619496


In [20]:
print(RPS1.mean(axis=1))

date
2022-03-02    0.205756
2022-03-03    0.238026
2022-03-04    0.184179
2022-03-07    0.203359
2022-03-08    0.204626
2022-03-09    0.186116
2022-03-10    0.213924
2022-03-11    0.171498
2022-03-14    0.188879
2022-03-15    0.187173
2022-03-16    0.225732
2022-03-17    0.229816
2022-03-18    0.225249
2022-03-21    0.208063
2022-03-22    0.218940
2022-03-23    0.211560
2022-03-24    0.192344
2022-03-25    0.205776
2022-03-28    0.227968
2022-03-29    0.216933
2022-03-30    0.221293
2022-03-31    0.218551
2022-04-01    0.272375
dtype: float64


In [26]:
#save results 
RPS1.to_excel('6_Mar_1_Apr-log_reg(baseline)-one_m_less.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('6_Mar_1_Apr-PROBABILITIES-log_reg(baseline)-one_m_less.xlsx')

#####Gradient Boosting (after feature selection and tuning to avoid overfitting)
***

In [21]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [22]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,21)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-01-31    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.1276021  0.19851893 0.20401765 0.20684662 0.2630147 ]
Date
2022-01-31    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.1820684  0.20942301 0.19482922 0.23154616 0.1821332 ]
Date
2022-01-31    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-03  159.814133  99.930481  183.961975  110.589905  126.775017   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-03  294.668579  163.559998  252.387024  173.579086  185.944336  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-03  115.603111  116.669998  303.419739  34.615841  89.954269   

                   UNH    URI           V        VRSK        XOM  
Date                                                              
2022-03-03  509.170868  354.0  225.529037  213.353729  81.485481  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-03-04  159.038071  99.08387  180.370453  110.371742  128.794907   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-04  294.787476  168.346497  248.984695  173.113022  186.431122  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-04  113.630821  121.860001  293.727295  34.894279  92.495735   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-04  506.618011  349.899994  226.903992  216.233383  81.524696  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-31  146.499664  96.603073  179.417999  113.822723  112.406982   

                   AMP   AMZN         AVB         AVY         AXP  ...  \
Date                                                               ...   
2022-03-31  266.332184  124.5  219.365616  177.753952  170.980133  ...   

                   PRU       PYPL          RE       ROL        ROST  \
Date                                                                  
2022-03-31  105.849098  91.529999  274.105255  33.01482  100.586761   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-31  497.807159  307.029999  210.753479  201.466431  84.710785  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-01  148.213638  97.046082  181.640396  114.437546  113.232849   

                  AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                   ...   
2022-04-01  269.58194  124.253502  220.645218  179.657928  171.468307  ...   

                   PRU       PYPL          RE       ROL       ROST  \
Date                                                                 
2022-04-01  107.137924  90.709999  276.180115  32.52755  99.723045   

                   UNH         URI           V        VRSK       XOM  
Date                                                                  
2022-04-01  496.287384  312.450012  207.794388  199.403824  86.45578  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \


,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-02,0.078246,0.235243,0.267511,0.120138,0.239041,0.300108,0.110671,0.230811,0.073982,0.124458,...,0.145365,0.121148,0.213374,0.252093,0.099074,0.071471,0.124301,0.254052,0.128772,0.091719
2022-03-03,0.137688,0.121689,0.265614,0.075057,0.130444,0.073096,0.251315,0.272287,0.092026,0.122966,...,0.271747,0.261817,0.260000,0.114527,0.270866,0.136304,0.201802,0.117213,0.105651,0.092577
2022-03-04,0.189366,0.106301,0.061368,0.106456,0.239041,0.075338,0.193872,0.072317,0.130674,0.122732,...,0.120811,0.173262,0.119026,0.084543,0.094403,0.220445,0.205511,0.071906,0.252144,0.091304
2022-03-07,0.189366,0.225982,0.120982,0.233224,0.146219,0.269099,0.213154,0.239098,0.100355,0.126027,...,0.079846,0.173262,0.125496,0.084778,0.144069,0.198161,0.202885,0.250288,0.064092,0.127118
2022-03-08,0.201412,0.250402,0.056449,0.116410,0.196816,0.293990,0.108022,0.270068,0.091714,0.140430,...,0.271747,0.121261,0.130024,0.213851,0.143725,0.099996,0.207599,0.234394,0.062548,0.213562
2022-03-09,0.101057,0.122660,0.057200,0.236271,0.234405,0.093585,0.194753,0.073674,0.149482,0.076911,...,0.195802,0.173262,0.075525,0.132098,0.136643,0.224159,0.201802,0.075650,0.128772,0.213562
2022-03-10,0.291518,0.077194,0.267445,0.262725,0.233436,0.101882,0.113136,0.072050,0.309564,0.255811,...,0.110287,0.121148,0.119116,0.219192,0.275114,0.066747,0.146194,0.073450,0.062176,0.239317
2022-03-11,0.291518,0.240667,0.056449,0.074835,0.233436,0.130180,0.140049,0.101226,0.091977,0.126027,...,0.144676,0.173312,0.262580,0.084543,0.275990,0.071471,0.242085,0.235625,0.129708,0.213562
2022-03-14,0.297734,0.250402,0.113860,0.258808,0.197878,0.228521,0.246058,0.142492,0.197368,0.226340,...,0.076140,0.173312,0.075525,0.132098,0.094172,0.071471,0.231824,0.126333,0.263054,0.130831


In [23]:
print(RPS2.mean(axis=1).mean())

0.1586364746016599


In [24]:
print(RPS2.mean(axis=1))

date
2022-03-02    0.162916
2022-03-03    0.163985
2022-03-04    0.151664
2022-03-07    0.153684
2022-03-08    0.154402
2022-03-09    0.152500
2022-03-10    0.167414
2022-03-11    0.156516
2022-03-14    0.160803
2022-03-15    0.155118
2022-03-16    0.156405
2022-03-17    0.164357
2022-03-18    0.159446
2022-03-21    0.156313
2022-03-22    0.161380
2022-03-23    0.160319
2022-03-24    0.155151
2022-03-25    0.155898
2022-03-28    0.159079
2022-03-29    0.162620
2022-03-30    0.162105
2022-03-31    0.155419
2022-04-01    0.161145
dtype: float64


In [25]:

#save results 
RPS2.to_excel('6_Mar_1_Apr-log_reg(with_feature_imp)-one_m_less.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('6_Mar_1_Apr-PROBABILITIES-log_reg(with_feature_imp)-one_m_less.xlsx')

####**For 3 Apr.-29 Apr.**

#####Features
***

In [27]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-04-01','2022-05-02') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [28]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#####Gradient Boosting (baseline) 
***

In [29]:
from sklearn.ensemble import GradientBoostingClassifier
baseline= GradientBoostingClassifier()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-02-28    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.05473146 0.05884415 0.19008785 0.33287869 0.36345785]
Date
2022-02-28    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.00185217 0.93579487 0.01206734 0.03798936 0.01229626]
Date
2022-02-28    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-30  145.518845  97.567833  180.449829  113.287216  109.800034   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-30  263.042786  124.281502  225.644547  179.092697  174.058624  ...   

                   PRU   PYPL          RE        ROL       ROST         UNH  \
Date                                                                          
2022-03-30  105.946739  87.93  271.417877  33.352924  99.047974  505.157806   

                   URI          V        VRSK        XOM  
Date                                                      
2022-03-30  316.519989  212.34761  203.319763  83.573593  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-31  146.499664  96.603073  179.417999  113.822723  112.406982   

                   AMP   AMZN         AVB         AVY         AXP  ...  \
Date                                                               ...   
2022-03-31  266.332184  124.5  219.365616  177.753952  170.980133  ...   

                   PRU       PYPL          RE       ROL        ROST  \
Date                                                                  
2022-03-31  105.849098  91.529999  274.105255  33.01482  100.586761   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-31  497.807159  307.029999  210.753479  201.466431  84.710785  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date          

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE       AMAT  \
Date                                                                    
2022-04-28  148.609924  102.762794  180.665848  112.593063  119.17524   

                   AMP      AMZN         AVB         AVY         AXP  ...  \
Date                                                                  ...   
2022-04-28  277.202972  115.1465  209.208252  172.161026  168.967667  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-04-28  105.03669  85.209999  283.079163  36.513096  86.678146   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-28  503.727386  297.459991  212.503616  178.010666  96.650909  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP        AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-29  146.004288  101.234802  175.96521  110.718826  116.990822   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-04-29  274.993225  120.209503  206.282059  171.862244  168.190582  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-29  105.016922  85.209999  280.802887  35.366138  84.404716   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-29  493.466309  298.179993  211.794876  174.294022  95.076218  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE      

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-29,0.484104,0.368281,0.104615,0.296291,0.157984,0.152471,0.219307,0.405677,0.195691,0.380732,...,0.176570,0.475157,0.247991,0.137434,0.284480,0.105937,0.237269,0.367660,0.193128,0.373283
2022-03-30,0.094070,0.390416,0.020660,0.430525,0.375183,0.090090,0.152631,0.027573,0.002918,0.602445,...,0.202156,0.171120,0.153869,0.353296,0.357208,0.553275,0.350362,0.012618,0.173958,0.128255
2022-03-31,0.342710,0.142965,0.188154,0.136909,0.167328,0.264425,0.232032,0.085109,0.180940,0.387554,...,0.155913,0.033232,0.148115,0.392940,0.315737,0.292304,0.399161,0.336347,0.382754,0.449427
2022-04-01,0.104259,0.126108,0.061983,0.188421,0.160003,0.037834,0.330874,0.314426,0.100777,0.288285,...,0.186879,0.178162,0.089152,0.395784,0.077253,0.695349,0.081969,0.112274,0.324623,0.164396
2022-04-04,0.135744,0.008198,0.408595,0.244146,0.616381,0.389472,0.199086,0.279965,0.026217,0.171850,...,0.026755,0.454736,0.337395,0.116082,0.401554,0.145723,0.105790,0.609779,0.000485,0.113254
2022-04-05,0.086764,0.147397,0.016073,0.115500,0.180749,0.256649,0.175784,0.285085,0.169342,0.377950,...,0.194061,0.531551,0.164954,0.380578,0.403678,0.280786,0.307332,0.294037,0.197172,0.213715
2022-04-06,0.330746,0.025202,0.099872,0.421722,0.014166,0.118943,0.174339,0.239762,0.124556,0.452855,...,0.172178,0.137497,0.255007,0.188950,0.287312,0.264794,0.182715,0.493506,0.195692,0.157913
2022-04-07,0.426362,0.303514,0.307349,0.321991,0.126361,0.047885,0.189512,0.290440,0.387488,0.062314,...,0.106047,0.138053,0.140053,0.092973,0.284987,0.153804,0.053788,0.003016,0.194393,0.108629
2022-04-08,0.249312,0.067574,0.079975,0.311111,0.120425,0.141144,0.120674,0.422255,0.015884,0.587253,...,0.121050,0.189395,0.176930,0.228104,0.070367,0.128630,0.111140,0.203935,0.185698,0.529754


In [30]:
print(RPS1.mean(axis=1).mean())

0.21574955094285472


In [31]:
print(RPS1.mean(axis=1))

date
2022-03-29    0.237390
2022-03-30    0.204463
2022-03-31    0.248446
2022-04-01    0.202232
2022-04-04    0.211421
2022-04-05    0.226406
2022-04-06    0.212439
2022-04-07    0.219424
2022-04-08    0.178366
2022-04-11    0.249522
2022-04-12    0.194330
2022-04-13    0.196749
2022-04-14    0.186726
2022-04-18    0.219975
2022-04-19    0.202887
2022-04-20    0.213031
2022-04-21    0.204178
2022-04-22    0.189912
2022-04-25    0.225129
2022-04-26    0.228790
2022-04-27    0.265884
2022-04-28    0.223062
2022-04-29    0.221475
dtype: float64


#####Gradient Boosting (after feature selection and tuning to avoid overfitting)
***

In [32]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [33]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,21)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-02-28    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.11964988 0.21105466 0.21527067 0.22330147 0.23072332]
Date
2022-02-28    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.18115857 0.20722676 0.1822235  0.235185   0.19420616]
Date
2022-02-28    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-30  145.518845  97.567833  180.449829  113.287216  109.800034   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-30  263.042786  124.281502  225.644547  179.092697  174.058624  ...   

                   PRU   PYPL          RE        ROL       ROST         UNH  \
Date                                                                          
2022-03-30  105.946739  87.93  271.417877  33.352924  99.047974  505.157806   

                   URI          V        VRSK        XOM  
Date                                                      
2022-03-30  316.519989  212.34761  203.319763  83.573593  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-31  146.499664  96.603073  179.417999  113.822723  112.406982   

                   AMP   AMZN         AVB         AVY         AXP  ...  \
Date                                                               ...   
2022-03-31  266.332184  124.5  219.365616  177.753952  170.980133  ...   

                   PRU       PYPL          RE       ROL        ROST  \
Date                                                                  
2022-03-31  105.849098  91.529999  274.105255  33.01482  100.586761   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-31  497.807159  307.029999  210.753479  201.466431  84.710785  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date          

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE       AMAT  \
Date                                                                    
2022-04-28  148.609924  102.762794  180.665848  112.593063  119.17524   

                   AMP      AMZN         AVB         AVY         AXP  ...  \
Date                                                                  ...   
2022-04-28  277.202972  115.1465  209.208252  172.161026  168.967667  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-04-28  105.03669  85.209999  283.079163  36.513096  86.678146   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-28  503.727386  297.459991  212.503616  178.010666  96.650909  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP        AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-29  146.004288  101.234802  175.96521  110.718826  116.990822   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-04-29  274.993225  120.209503  206.282059  171.862244  168.190582  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-29  105.016922  85.209999  280.802887  35.366138  84.404716   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-29  493.466309  298.179993  211.794876  174.294022  95.076218  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE      

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-29,0.309479,0.133236,0.108971,0.220210,0.145666,0.097153,0.202353,0.271103,0.282521,0.123334,...,0.079855,0.168022,0.078310,0.119641,0.201595,0.077573,0.235225,0.112272,0.132156,0.136773
2022-03-30,0.089420,0.124691,0.056089,0.128752,0.205080,0.097153,0.090873,0.258997,0.101761,0.252692,...,0.079855,0.272019,0.111593,0.128530,0.265631,0.295685,0.217518,0.116653,0.132153,0.134436
2022-03-31,0.089420,0.082228,0.109255,0.076462,0.115320,0.097478,0.107308,0.072756,0.131167,0.123077,...,0.104162,0.168022,0.078310,0.246531,0.201595,0.149692,0.235225,0.233871,0.272423,0.224746
2022-04-01,0.309479,0.124691,0.116654,0.079467,0.198663,0.211790,0.202353,0.272215,0.061560,0.121334,...,0.135316,0.123305,0.078310,0.218306,0.137656,0.295685,0.094840,0.126344,0.272423,0.215327
2022-04-04,0.194821,0.099666,0.257575,0.109883,0.236796,0.283950,0.194248,0.104375,0.061560,0.125203,...,0.078054,0.175624,0.212176,0.119641,0.108842,0.097604,0.217518,0.233871,0.059765,0.127750
2022-04-05,0.091495,0.110881,0.056048,0.110020,0.116464,0.141252,0.194248,0.272215,0.101389,0.250228,...,0.126337,0.175624,0.206092,0.218306,0.201595,0.211885,0.203667,0.116653,0.131990,0.216591
2022-04-06,0.094441,0.237608,0.116596,0.267454,0.236796,0.075503,0.194248,0.129809,0.131167,0.125222,...,0.116623,0.121434,0.078482,0.119641,0.108842,0.073905,0.125242,0.233871,0.131990,0.230537
2022-04-07,0.094441,0.124938,0.108971,0.224112,0.202932,0.075503,0.091709,0.261143,0.231481,0.121323,...,0.126337,0.125432,0.078482,0.246531,0.204556,0.073905,0.234264,0.077854,0.104912,0.129890
2022-04-08,0.094413,0.233466,0.059044,0.112772,0.222008,0.075503,0.194248,0.230851,0.131167,0.123079,...,0.078054,0.175624,0.111593,0.218306,0.108842,0.097710,0.212585,0.250412,0.131990,0.219296


In [34]:
print(RPS2.mean(axis=1).mean())

0.15913146086104896


In [35]:
print(RPS2.mean(axis=1))

date
2022-03-29    0.162030
2022-03-30    0.163757
2022-03-31    0.155837
2022-04-01    0.161412
2022-04-04    0.153446
2022-04-05    0.153240
2022-04-06    0.158932
2022-04-07    0.158867
2022-04-08    0.148024
2022-04-11    0.166518
2022-04-12    0.160470
2022-04-13    0.152528
2022-04-14    0.160475
2022-04-18    0.161081
2022-04-19    0.168811
2022-04-20    0.164070
2022-04-21    0.152813
2022-04-22    0.154263
2022-04-25    0.160879
2022-04-26    0.160084
2022-04-27    0.160426
2022-04-28    0.162285
2022-04-29    0.159776
dtype: float64


####**For 1 May-27 May**

#####Features
***

In [36]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-05-01','2022-05-28') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [37]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#####Gradient Boosting (baseline) 
***

In [38]:
from sklearn.ensemble import GradientBoostingClassifier
baseline= GradientBoostingClassifier()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-03-28    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.03906299 0.2757924  0.29522409 0.09725838 0.29266214]
Date
2022-03-28    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.01924087 0.030068   0.33725282 0.58961532 0.02382299]
Date
2022-03-28    2
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV         AEP         AIZ        ALLE       AMAT  \
Date                                                                    
2022-04-28  148.609924  102.762794  180.665848  112.593063  119.17524   

                   AMP      AMZN         AVB         AVY         AXP  ...  \
Date                                                                  ...   
2022-04-28  277.202972  115.1465  209.208252  172.161026  168.967667  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-04-28  105.03669  85.209999  283.079163  36.513096  86.678146   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-28  503.727386  297.459991  212.503616  178.010666  96.650909  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-29  146.004288  101.234802  175.96521  110.718826  116.990822   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-04-29  274.993225  120.209503  206.282059  171.862244  168.190582  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-29  105.016922  85.209999  280.802887  35.366138  84.404716   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-29  493.466309  298.179993  211.794876  174.294022  95.076218  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-26  151.076859  94.338966  172.290344  97.975998  94.777618   

                   AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                   ...   
2022-05-26  242.633484  107.400002  195.470001  163.45636  141.659866  ...   

                  PRU   PYPL          RE        ROL       ROST        UNH  \
Date                                                                        
2022-05-26  96.546379  71.82  278.745331  34.418652  72.525093  506.84964   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-26  247.990005  197.769714  168.658813  90.619514  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-27  152.711563  94.626709  170.169083  97.966034  91.705482   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-27  238.890869  108.919998  195.080002  163.336853  138.949997  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-27  95.212051  71.470001  276.866699  34.877438  71.558487   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-05-27  514.096924  241.070007  199.147278  171.034866  87.272049  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-04-27,0.201505,0.657826,0.243814,0.241718,0.324123,0.141122,0.481479,0.084886,0.118774,0.129291,...,0.232796,0.095430,0.066030,0.727348,0.493809,0.262792,0.208681,0.004723,0.398560,0.261508
2022-04-28,0.391804,0.140203,0.139826,0.162874,0.095293,0.139668,0.654447,0.195944,0.125919,0.102199,...,0.074966,0.761430,0.003993,0.183373,0.550529,0.258194,0.672870,0.278699,0.195255,0.170151
2022-04-29,0.106920,0.033757,0.351432,0.160531,0.352811,0.241482,0.348349,0.002743,0.283151,0.251882,...,0.101739,0.001740,0.269402,0.078387,0.596642,0.150985,0.255922,0.128300,0.042840,0.067291
2022-05-02,0.337109,0.455695,0.076938,0.074116,0.152026,0.224415,0.165356,0.390509,0.313098,0.180728,...,0.000564,0.761981,0.002121,0.404518,0.349440,0.263382,0.121457,0.151388,0.374419,0.038972
2022-05-03,0.348447,0.061662,0.385472,0.045025,0.102958,0.013284,0.000419,0.573572,0.238202,0.137419,...,0.230977,0.002675,0.141045,0.262134,0.127809,0.123267,0.111810,0.094722,0.273694,0.199788
2022-05-04,0.207762,0.360364,0.280106,0.078127,0.099237,0.073106,0.337655,0.438432,0.160342,0.434922,...,0.251875,0.244205,0.323865,0.143674,0.251822,0.354502,0.529285,0.007766,0.156187,0.116605
2022-05-05,0.457175,0.358841,0.019601,0.235297,0.224381,0.212746,0.044430,0.112980,0.181429,0.086684,...,0.001471,0.732856,0.084080,0.151397,0.520683,0.112816,0.240981,0.103955,0.065151,0.085164
2022-05-06,0.584596,0.222448,0.033229,0.404138,0.198688,0.125710,0.094370,0.592258,0.135792,0.376634,...,0.095848,0.308625,0.638945,0.553842,0.265521,0.103046,0.184611,0.150618,0.166858,0.270235
2022-05-09,0.213884,0.139249,0.260027,0.046533,0.060997,0.104371,0.167984,0.271264,0.413696,0.246458,...,0.190132,0.019435,0.135647,0.214687,0.162220,0.182020,0.354027,0.514562,0.133653,0.195962


In [39]:
print(RPS1.mean(axis=1).mean())

0.2089648125389678


In [40]:
print(RPS1.mean(axis=1))

date
2022-04-27    0.252935
2022-04-28    0.217047
2022-04-29    0.175689
2022-05-02    0.239918
2022-05-03    0.158564
2022-05-04    0.227238
2022-05-05    0.208997
2022-05-06    0.245877
2022-05-09    0.237014
2022-05-10    0.199238
2022-05-11    0.187192
2022-05-12    0.222236
2022-05-13    0.246168
2022-05-16    0.186899
2022-05-17    0.216345
2022-05-18    0.177481
2022-05-19    0.189958
2022-05-20    0.198861
2022-05-23    0.226614
2022-05-24    0.184710
2022-05-25    0.167724
2022-05-26    0.217794
2022-05-27    0.221693
dtype: float64


#####Gradient Boosting (after feature selection and tuning to avoid overfitting)
***

In [41]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [42]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,21)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-03-28    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.14525879 0.17924785 0.21333649 0.21552947 0.24662739]
Date
2022-03-28    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16930319 0.20797918 0.18327108 0.2503069  0.18913964]
Date
2022-03-28    2
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV         AEP         AIZ        ALLE       AMAT  \
Date                                                                    
2022-04-28  148.609924  102.762794  180.665848  112.593063  119.17524   

                   AMP      AMZN         AVB         AVY         AXP  ...  \
Date                                                                  ...   
2022-04-28  277.202972  115.1465  209.208252  172.161026  168.967667  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-04-28  105.03669  85.209999  283.079163  36.513096  86.678146   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-28  503.727386  297.459991  212.503616  178.010666  96.650909  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-29  146.004288  101.234802  175.96521  110.718826  116.990822   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-04-29  274.993225  120.209503  206.282059  171.862244  168.190582  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-29  105.016922  85.209999  280.802887  35.366138  84.404716   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-29  493.466309  298.179993  211.794876  174.294022  95.076218  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-26  151.076859  94.338966  172.290344  97.975998  94.777618   

                   AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                   ...   
2022-05-26  242.633484  107.400002  195.470001  163.45636  141.659866  ...   

                  PRU   PYPL          RE        ROL       ROST        UNH  \
Date                                                                        
2022-05-26  96.546379  71.82  278.745331  34.418652  72.525093  506.84964   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-26  247.990005  197.769714  168.658813  90.619514  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-27  152.711563  94.626709  170.169083  97.966034  91.705482   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-27  238.890869  108.919998  195.080002  163.336853  138.949997  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-27  95.212051  71.470001  276.866699  34.877438  71.558487   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-05-27  514.096924  241.070007  199.147278  171.034866  87.272049  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-04-27,0.287409,0.255120,0.125255,0.122065,0.200064,0.139246,0.261216,0.107245,0.126984,0.125607,...,0.075665,0.285583,0.077644,0.254229,0.219211,0.287704,0.238606,0.072028,0.123851,0.265454
2022-04-28,0.144178,0.130185,0.282619,0.118151,0.146016,0.074455,0.261216,0.124907,0.127207,0.127555,...,0.113937,0.285583,0.072101,0.234843,0.219211,0.287122,0.231239,0.124657,0.255381,0.151324
2022-04-29,0.078990,0.109531,0.245021,0.263525,0.216177,0.275915,0.263032,0.107635,0.275486,0.117929,...,0.075665,0.147080,0.134014,0.123347,0.217846,0.067469,0.127664,0.077666,0.261572,0.181477
2022-05-02,0.144088,0.255488,0.121908,0.073191,0.139637,0.074455,0.263032,0.135165,0.126984,0.124455,...,0.251028,0.285583,0.072101,0.234843,0.111889,0.287122,0.135317,0.124657,0.123851,0.251531
2022-05-03,0.202424,0.106133,0.253195,0.235849,0.216177,0.139246,0.204629,0.277176,0.126984,0.115037,...,0.251028,0.133972,0.115057,0.230946,0.088682,0.137474,0.232069,0.077490,0.124561,0.196387
2022-05-04,0.292366,0.128667,0.105509,0.073282,0.142627,0.096567,0.263032,0.257967,0.126984,0.117929,...,0.117238,0.134208,0.211615,0.090100,0.217846,0.103377,0.228468,0.077490,0.059550,0.151324
2022-05-05,0.202424,0.124920,0.103877,0.115411,0.135464,0.074455,0.204629,0.243515,0.063793,0.074310,...,0.063843,0.302640,0.077187,0.124664,0.217846,0.101701,0.233812,0.115517,0.124561,0.181477
2022-05-06,0.201972,0.124920,0.125255,0.240428,0.216351,0.139246,0.086184,0.274451,0.103695,0.109701,...,0.073558,0.134208,0.273858,0.144244,0.253207,0.102995,0.209281,0.124657,0.261572,0.196387
2022-05-09,0.144178,0.079645,0.102970,0.111897,0.216351,0.139246,0.204629,0.124907,0.135309,0.244612,...,0.122491,0.147080,0.114698,0.124664,0.126978,0.215487,0.110413,0.230705,0.059550,0.099240


In [43]:
print(RPS2.mean(axis=1).mean())

0.15862303585253285


In [44]:
print(RPS2.mean(axis=1))

date
2022-04-27    0.163247
2022-04-28    0.163335
2022-04-29    0.158083
2022-05-02    0.167362
2022-05-03    0.149690
2022-05-04    0.162282
2022-05-05    0.151935
2022-05-06    0.165384
2022-05-09    0.158933
2022-05-10    0.150359
2022-05-11    0.152630
2022-05-12    0.163908
2022-05-13    0.165472
2022-05-16    0.150994
2022-05-17    0.165567
2022-05-18    0.151436
2022-05-19    0.156092
2022-05-20    0.161085
2022-05-23    0.166170
2022-05-24    0.152006
2022-05-25    0.152648
2022-05-26    0.157578
2022-05-27    0.162133
dtype: float64


####For the whole period with one month less training 
***

In [46]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-03-01','2022-05-28') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [47]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#####Gradient Boosting (baseline) 
***

In [48]:
from sklearn.ensemble import GradientBoostingClassifier
baseline= GradientBoostingClassifier()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-01-31    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.12451265 0.06802864 0.21476629 0.30032585 0.29236657]
Date
2022-01-31    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.02749721 0.08418458 0.35048786 0.49821568 0.03961467]
Date
2022-01-31    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-03  159.814133  99.930481  183.961975  110.589905  126.775017   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-03  294.668579  163.559998  252.387024  173.579086  185.944336  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-03  115.603111  116.669998  303.419739  34.615841  89.954269   

                   UNH    URI           V        VRSK        XOM  
Date                                                              
2022-03-03  509.170868  354.0  225.529037  213.353729  81.485481  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-03-04  159.038071  99.08387  180.370453  110.371742  128.794907   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-04  294.787476  168.346497  248.984695  173.113022  186.431122  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-04  113.630821  121.860001  293.727295  34.894279  92.495735   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-04  506.618011  349.899994  226.903992  216.233383  81.524696  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-26  151.076859  94.338966  172.290344  97.975998  94.777618   

                   AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                   ...   
2022-05-26  242.633484  107.400002  195.470001  163.45636  141.659866  ...   

                  PRU   PYPL          RE        ROL       ROST        UNH  \
Date                                                                        
2022-05-26  96.546379  71.82  278.745331  34.418652  72.525093  506.84964   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-26  247.990005  197.769714  168.658813  90.619514  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-27  152.711563  94.626709  170.169083  97.966034  91.705482   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-27  238.890869  108.919998  195.080002  163.336853  138.949997  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-27  95.212051  71.470001  276.866699  34.877438  71.558487   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-05-27  514.096924  241.070007  199.147278  171.034866  87.272049  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-02,0.137302,0.090944,0.303174,0.085041,0.303978,0.548108,0.272005,0.004715,0.373465,0.190281,...,0.577992,0.226650,0.189543,0.560663,0.008986,0.040034,0.117563,0.373404,0.120199,0.133125
2022-03-03,0.079842,0.333812,0.313133,0.314670,0.117350,0.004537,0.362518,0.107617,0.080374,0.205306,...,0.557973,0.176286,0.442738,0.195002,0.332104,0.169605,0.252647,0.164616,0.110699,0.028988
2022-03-04,0.115404,0.012261,0.155122,0.102211,0.361525,0.101142,0.208082,0.125184,0.256600,0.174059,...,0.146719,0.084447,0.120223,0.085128,0.092827,0.402547,0.784742,0.142725,0.214723,0.014534
2022-03-07,0.128604,0.200216,0.220866,0.370297,0.095313,0.350456,0.285399,0.301869,0.115819,0.274834,...,0.126342,0.006371,0.167218,0.093614,0.266265,0.482623,0.181746,0.177053,0.189393,0.134913
2022-03-08,0.016870,0.521174,0.017786,0.130632,0.579987,0.203437,0.117287,0.355670,0.172175,0.270842,...,0.595274,0.161801,0.190884,0.282169,0.101089,0.220666,0.340037,0.143728,0.073314,0.010073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-23,0.179625,0.412366,0.124678,0.271734,0.490823,0.042451,0.318476,0.384954,0.405850,0.142248,...,0.262072,0.184171,0.008393,0.150658,0.033445,0.634415,0.213327,0.184373,0.221826,0.465364
2022-05-24,0.144320,0.179299,0.079441,0.517498,0.143604,0.202424,0.111984,0.070239,0.131091,0.375517,...,0.341986,0.026963,0.178800,0.012691,0.329832,0.034319,0.207857,0.028962,0.089913,0.300073
2022-05-25,0.118730,0.197476,0.251103,0.012735,0.187362,0.100900,0.298995,0.054551,0.192522,0.125526,...,0.278876,0.001609,0.197323,0.136666,0.185902,0.120191,0.151385,0.096081,0.556862,0.112556


In [49]:
 print(RPS1.mean(axis=1).mean())

0.2126127151711454


In [50]:
 print(RPS1.mean(axis=1))

date
2022-03-02    0.205905
2022-03-03    0.239329
2022-03-04    0.194105
2022-03-07    0.211493
2022-03-08    0.200712
                ...   
2022-05-23    0.218454
2022-05-24    0.188887
2022-05-25    0.173204
2022-05-26    0.189315
2022-05-27    0.199748
Length: 62, dtype: float64


In [52]:
#for 1st April
print(RPS1.iloc[0].mean())
#for 29 April 
print(RPS1.iloc[19].mean())
#for 27 May 
print(RPS1.iloc[39].mean())

0.20590530329634563
0.20672808582962646
0.2472151447676015


#####Gradient Boosting (after feature selection and tuning to avoid overfitting)
***

In [50]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [51]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,21)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-01-31    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.12656504 0.193119   0.23261355 0.19945008 0.24825234]
Date
2022-01-31    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16864415 0.20787495 0.19162416 0.2399934  0.19186334]
Date
2022-01-31    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-03  159.814133  99.930481  183.961975  110.589905  126.775017   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-03  294.668579  163.559998  252.387024  173.579086  185.944336  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-03  115.603111  116.669998  303.419739  34.615841  89.954269   

                   UNH    URI           V        VRSK        XOM  
Date                                                              
2022-03-03  509.170868  354.0  225.529037  213.353729  81.485481  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-03-04  159.038071  99.08387  180.370453  110.371742  128.794907   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-04  294.787476  168.346497  248.984695  173.113022  186.431122  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-04  113.630821  121.860001  293.727295  34.894279  92.495735   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-04  506.618011  349.899994  226.903992  216.233383  81.524696  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-26  151.076859  94.338966  172.290344  97.975998  94.777618   

                   AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                   ...   
2022-05-26  242.633484  107.400002  195.470001  163.45636  141.659866  ...   

                  PRU   PYPL          RE        ROL       ROST        UNH  \
Date                                                                        
2022-05-26  96.546379  71.82  278.745331  34.418652  72.525093  506.84964   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-26  247.990005  197.769714  168.658813  90.619514  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-27  152.711563  94.626709  170.169083  97.966034  91.705482   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-27  238.890869  108.919998  195.080002  163.336853  138.949997  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-27  95.212051  71.470001  276.866699  34.877438  71.558487   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-05-27  514.096924  241.070007  199.147278  171.034866  87.272049  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-02,0.078070,0.229347,0.252190,0.111389,0.237311,0.322097,0.102688,0.240900,0.072438,0.120548,...,0.150565,0.131274,0.204357,0.271226,0.091548,0.071663,0.126987,0.260548,0.120551,0.089555
2022-03-03,0.148515,0.123624,0.272214,0.074540,0.130840,0.078817,0.263648,0.255907,0.103691,0.124068,...,0.283215,0.287151,0.278163,0.110993,0.277317,0.143177,0.208017,0.121396,0.115845,0.086392
2022-03-04,0.183524,0.103530,0.060652,0.113533,0.239282,0.079407,0.186181,0.070816,0.126435,0.119112,...,0.112995,0.149702,0.118514,0.086752,0.092176,0.208084,0.219097,0.076846,0.266495,0.086392
2022-03-07,0.183524,0.219814,0.110469,0.235623,0.154677,0.298379,0.202289,0.247230,0.103228,0.123640,...,0.077039,0.149702,0.132547,0.088138,0.150608,0.210443,0.205451,0.241473,0.061015,0.126643
2022-03-08,0.196028,0.259390,0.055699,0.111442,0.219931,0.322097,0.098164,0.263114,0.098067,0.134109,...,0.283215,0.131274,0.132547,0.219185,0.152101,0.098450,0.205451,0.223358,0.061015,0.206812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-23,0.147443,0.255752,0.115063,0.077077,0.148668,0.085539,0.160312,0.266444,0.295299,0.077539,...,0.220182,0.304643,0.115951,0.086752,0.137659,0.297954,0.222618,0.130521,0.265280,0.141552
2022-05-24,0.104455,0.229347,0.059776,0.124808,0.120831,0.149844,0.184094,0.240900,0.126435,0.244877,...,0.136178,0.133157,0.083771,0.086752,0.204639,0.204900,0.222618,0.212985,0.098413,0.195791
2022-05-25,0.096976,0.229347,0.262079,0.107300,0.154384,0.078870,0.184094,0.104301,0.093577,0.077515,...,0.220182,0.135890,0.115951,0.121446,0.204639,0.204900,0.103656,0.106606,0.265280,0.208132


In [53]:
print(RPS2.mean(axis=1).mean())

0.15870982236610018


In [54]:
print(RPS2.mean(axis=1))

date
2022-03-02    0.163247
2022-03-03    0.168133
2022-03-04    0.150549
2022-03-07    0.153073
2022-03-08    0.154191
                ...   
2022-05-23    0.165457
2022-05-24    0.153124
2022-05-25    0.149059
2022-05-26    0.159871
2022-05-27    0.158915
Length: 62, dtype: float64


In [55]:
#for 1st April
print(RPS2.iloc[0].mean())
#for 29 April 
print(RPS2.iloc[19].mean())
#for 27 May 
print(RPS2.iloc[39].mean())

0.16324703527865744
0.15959658823341918
0.16287895866326307
